In [3]:
# extracting the compessed Dataset
from zipfile import ZipFile
dataset = '/content/drive/MyDrive/mimic-iv-clinical-database-demo-on-fhir-2.0.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [6]:
import json
import pandas as pd

def load_json(file_path):
    with open(file_path, 'r') as file:
        return [json.loads(line) for line in file]

patients = load_json('/content/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/Patient.ndjson')
conditions = load_json('/content/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/Condition.ndjson')
encounters = load_json('/content/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/Encounter.ndjson')
icu_encounters = load_json('/content/mimic-iv-clinical-database-demo-on-fhir-2.0/mimic-fhir/EncounterICU.ndjson')

patient_conditions = {}

for condition in conditions:
    patient_id = condition['subject']['reference'].split('/')[1]
    if patient_id not in patient_conditions:
        patient_conditions[patient_id] = []
    patient_conditions[patient_id].append(condition)

def find_start_time(encounter_id):
    for encounter in encounters:
        if encounter['id'] == encounter_id:
            return encounter['period']['start']
    for icu_encounter in icu_encounters:
        if icu_encounter['id'] == encounter_id:
            return icu_encounter['period']['start']

csv_rows = []

for patient_id, conditions_list in patient_conditions.items():
    for condition in conditions_list:
        timestamp = find_start_time(condition['encounter']['reference'].split('/')[1])
        code = condition['code']['coding'][0]['code']
        description = condition['code']['coding'][0]['display']
        csv_rows.append({'pid': patient_id, 'time': timestamp, 'code': code, 'description': description})

df = pd.DataFrame(csv_rows)

df.to_csv('patient_conditions.csv', index=False)
